In [54]:
using Oceananigans 
using Oceananigans.Units
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CUDA: CuArray
using CairoMakie 
using NCDatasets
using Printf

In [105]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L_front/H
Ro = 0.08            # Rossby number (defines M^2)

# Derived parameters
H_front = L_front/aspect_ratio
M² = (Ro^2*f^2*L_front)/H_front
Δb = M²*L_front
κh = 1e-4
κv = 1e-4

filename = "periodic_geostrophic_adjustment_kh1e-2"

"periodic_geostrophic_adjustment_kh1e-2"

In [106]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), 
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [107]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=0.0001), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=0.0001))

In [108]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = (:b,:T),
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b=0.0001, T=0.0001))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b=0.0001, T=0.0001))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [109]:
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/20)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [110]:
simulation = Simulation(model, Δt=20minutes, stop_time=5days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [111]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [112]:

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(50))

In [113]:
# Output setup
u, v, w = model.velocities
b = model.tracers.b
wc = Field(@at (Center, Center, Center) model.velocities.w)
T = model.tracers.T

#For Lagrangian filtering
simulation.output_writers[:jld2fields] = JLD2Writer(
    model, (; b, u, v, w, wc,T), filename=filename * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


#for python visualisation
rm(filename * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, (; b, u, v, w, wc,T), filename=filename * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)
    

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_geostrophic_adjustment_kh1e-2.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 6 outputs: (v, w, T, wc, b, u)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 27.5 KiB

In [114]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[00.00%] i: 0, t: 0 seconds, wall time: 1.358 seconds, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info: Running the simulation...
[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (39.386 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (3.444 ms).


[09.78%] i: 50, t: 11.733 hours, wall time: 328.582 ms, max(u): (1.646e-02, 2.908e-02, 2.868e-04) m/s, next Δt: 11 minutes
[14.35%] i: 100, t: 17.218 hours, wall time: 230.144 ms, max(u): (2.560e-03, 9.525e-04, 3.193e-05) m/s, next Δt: 7.179 minutes
[19.07%] i: 150, t: 22.880 hours, wall time: 209.889 ms, max(u): (1.765e-02, 2.644e-02, 3.078e-04) m/s, next Δt: 5.865 minutes
[23.29%] i: 200, t: 1.164 days, wall time: 208.807 ms, max(u): (1.090e-02, 3.503e-02, 2.031e-04) m/s, next Δt: 7.807 minutes
[27.87%] i: 250, t: 1.394 days, wall time: 215.417 ms, max(u): (1.014e-02, 4.462e-03, 1.696e-04) m/s, next Δt: 6.731 minutes
[32.73%] i: 300, t: 1.636 days, wall time: 215.978 ms, max(u): (1.838e-02, 1.942e-02, 3.523e-04) m/s, next Δt: 6.042 minutes
[36.97%] i: 350, t: 1.849 days, wall time: 203.859 ms, max(u): (5.693e-03, 3.730e-02, 1.063e-04) m/s, next Δt: 7.311 minutes
[41.67%] i: 400, t: 2.083 days, wall time: 195.796 ms, max(u): (1.409e-02, 9.084e-03, 2.870e-04) m/s, next Δt: 6.924 minute

[ Info: Simulation is stopping after running for 4.512 seconds.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.
[ Info: Simulation completed in 4.533 seconds
